In [2]:
import clickhouse_connect
import duckdb
import pandas as pd
import os

In [3]:
# Configurações ClickHouse
ch_host = "gowlim80fx.us-east1.gcp.clickhouse.cloud"
ch_port = 8443
ch_username = "default"
ch_password = "JBhX.Pm6OJVO6"
ch_secure = True

# Diretório onde os .duckdb serão salvos
export_dir = "duckdb_exports"
os.makedirs(export_dir, exist_ok=True)

In [5]:
# Nome da base DuckDB a ser usada (você pode mudar se quiser outro arquivo)
duckdb_file = os.path.join(export_dir, "default.duckdb")

In [ ]:
# Conectar ao ClickHouse
client = clickhouse_connect.get_client(
    host=ch_host,
    port=ch_port,
    username=ch_username,
    password=ch_password,
    secure=ch_secure
)

# Obter bancos de dados (ignorar internos)
databases = client.query("SHOW DATABASES").result_rows
databases = [db[0] for db in databases if db[0] not in ["system", "information_schema"]]

# Tabelas específicas para exportação
tables_to_export = {"commits", "pull_requests"}

for db in databases:
    print(f"📦 Processando banco: {db}")
    client.command(f"USE {db}")
    tables = client.query("SHOW TABLES").result_rows
    tables = [t[0] for t in tables if t[0] in tables_to_export]  # Filtrar apenas as tabelas desejadas

    if not tables:
        print(f"⚠️ Banco {db} não possui as tabelas desejadas.")
        continue

    # Criar/conectar ao arquivo DuckDB
    db_path = os.path.join(export_dir, f"{db}.duckdb")
    con = duckdb.connect(db_path)

    for table in tables:
        print(f"🔄 Exportando tabela: {db}.{table}")
        try:
            df = client.query_df(f"SELECT * FROM {table}")
            if df.empty:
                print(f"⚠️ Tabela {table} vazia.")
                continue

            # Substituir tabela se já existir
            con.execute(f"DROP TABLE IF EXISTS {table}")
            con.register("df_temp", df)
            con.execute(f"CREATE TABLE {table} AS SELECT * FROM df_temp")
            con.unregister("df_temp")

            print(f"✅ Tabela {table} salva no {db}.duckdb")
        except Exception as e:
            print(f"❌ Erro ao exportar {table}: {e}")

    con.close()

print("🏁 Exportação concluída. Arquivos DuckDB salvos em 'duckdb_exports/'")

📦 Processando banco: INFORMATION_SCHEMA
⚠️ Banco INFORMATION_SCHEMA não possui as tabelas desejadas.
📦 Processando banco: default
🔄 Exportando tabela: default.commits
✅ Tabela commits salva no default.duckdb
🔄 Exportando tabela: default.pull_requests
✅ Tabela pull_requests salva no default.duckdb
🏁 Exportação concluída. Arquivos DuckDB salvos em 'duckdb_exports/'


In [7]:
# Tabela alvo
target_table = "repositories"

# Conectar ao ClickHouse
client = clickhouse_connect.get_client(
    host=ch_host,
    port=ch_port,
    username=ch_username,
    password=ch_password,
    secure=ch_secure
)

# Procurar a tabela repositories entre os bancos disponíveis
databases = client.query("SHOW DATABASES").result_rows
databases = [db[0] for db in databases if db[0] not in ["system", "information_schema"]]

found = False

for db in databases:
    print(f"🔍 Verificando banco: {db}")
    client.command(f"USE {db}")
    tables = client.query("SHOW TABLES").result_rows
    table_names = [t[0] for t in tables]
    
    if target_table in table_names:
        print(f"✅ Tabela '{target_table}' encontrada em: {db}")
        found = True
        df = client.query_df(f"SELECT * FROM {target_table}")
        
        if df.empty:
            print("⚠️ A tabela 'repositories' está vazia.")
            break

        con = duckdb.connect(duckdb_file)
        con.execute(f"DROP TABLE IF EXISTS {target_table}")
        con.register("df_temp", df)
        con.execute(f"CREATE TABLE {target_table} AS SELECT * FROM df_temp")
        con.unregister("df_temp")
        con.close()

        print(f"💾 Tabela '{target_table}' exportada com sucesso para {duckdb_file}")
        break

if not found:
    print("❌ Tabela 'repositories' não encontrada em nenhum banco.")

print("🏁 Finalizado.")

🔍 Verificando banco: INFORMATION_SCHEMA
🔍 Verificando banco: default
✅ Tabela 'repositories' encontrada em: default
💾 Tabela 'repositories' exportada com sucesso para duckdb_exports/default.duckdb
🏁 Finalizado.
